In [7]:
import os
from pathlib import Path
import json
import glob
import h5py
import numpy as np
from tqdm.notebook import tqdm

In [8]:
outdir = '../../data_prods/'
#data_dir = '/home/ryan/Data'
data_dir = 'D:/'
subdir_list = [os.path.join(data_dir, o) for o in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir,o))]

print(subdir_list)

['D:/run_2cDM_L3N128_HY_power00_sigma1', 'D:/run_2cDM_L3N256_HY_power00_sigma1', 'D:/run_2cDM_L3N256_HY_power00_sigma1_dir_0', 'D:/run_2cDM_L3N256_HY_power00_sigma1_dir_1', 'D:/run_2cDM_L3N256_HY_power00_sigma1_dir_2', 'D:/run_2cDM_L3N256_HY_power00_sigma1_dir_3', 'D:/run_2cDM_L3N256_HY_power00_sigma1_dir_4', 'D:/run_2cDM_L3N256_HY_power00_sigma1_dir_5', 'D:/run_2cDM_L3N256_HY_power00_sigma1_dir_6', 'D:/run_2cDM_L3N256_HY_power00_sigma1_dir_7', 'D:/run_2cDM_L3N256_HY_power00_sigma1_dir_8', 'D:/run_2cDM_L3N256_HY_power00_sigma1_dir_9', 'D:/run_2cDM_L3N256_HY_powerm2m2_sigma1', 'D:/run_2cDM_L3N256_HY_powerm2m2_sigma1_dir_0', 'D:/run_2cDM_L3N256_HY_powerm2m2_sigma1_dir_1', 'D:/run_2cDM_L3N256_HY_powerm2m2_sigma1_dir_2', 'D:/run_2cDM_L3N256_HY_powerm2m2_sigma1_dir_3', 'D:/run_2cDM_L3N256_HY_powerm2m2_sigma1_dir_4', 'D:/run_2cDM_L3N256_HY_powerm2m2_sigma1_dir_5', 'D:/run_2cDM_L3N256_HY_powerm2m2_sigma1_dir_6', 'D:/run_2cDM_L3N256_HY_powerm2m2_sigma1_dir_7', 'D:/run_2cDM_L3N256_HY_powerm2m2_

In [9]:
rho_crit = 2.7754*10**11 # solar mass / Mpc^3

In [10]:

def parsePower(powerType):
    stripped = powerType[5:]
    negFlag = False
    powers = [] 
    for char in stripped:
        if char == 'm':
            negFlag = True
            continue
        if negFlag:
            powers.append(-int(char))
            negFlag = False
        else:
            powers.append(int(char))

    return powers

def parseSigma(sigmaType):
    return float(sigmaType[5:])


In [11]:
def get_profiles(f, nbins=1000):

    Subhalo=f.get('Subhalo')
    SubhaloMass=np.array(Subhalo['SubhaloMass'])
    SubhaloVMax=np.array(Subhalo['SubhaloVmax']) #km/s
    MaxVel=np.absolute(SubhaloVMax)
    SubhaloMass=10**10*SubhaloMass #converting to solar masses
    SubhaloN = Subhalo['SubhaloLen'][()]
    enough = SubhaloN > 30

    smass = SubhaloMass[enough]
    svmax = MaxVel[enough]

    Mass_Bins = np.geomspace(np.amin(smass), np.amax(smass), num=nbins)
    MaxVel_Bin = np.geomspace(np.amin(svmax), np.amax(svmax), num=nbins)

    N_M = np.array([ (smass > mbin).sum() for mbin in Mass_Bins ])
    N_V = np.array([ (svmax > vbin).sum() for vbin in MaxVel_Bin])
    
    return (Mass_Bins, N_M), (MaxVel_Bin, N_V)

In [12]:
"""
TODO:

Record OmegaB and OmegaStar for each snapshot in hydro runs
"""
for subdir in tqdm(subdir_list):

    #run name is just folder name
    run_name = subdir.split('/')[-1]

    #reject folders that aren't runs
    if 'run' not in run_name:
        continue

    fpath = os.path.join(outdir, run_name)
    try:
        os.mkdir(fpath)
    except:
        print(f'{fpath} already exists!')
    print("Working on ", run_name)

    # read info from snapshots/fof
    snaps = sorted(glob.glob(subdir + '/snap*'))
    fofs = sorted(glob.glob(subdir + '/fof*'))

    if len(snaps) == 0:
        print(f'No snapshots for run {run_name}...')
        continue

    #get info from filename
    splitted = run_name.split('_')

    dm_type = splitted[1]

    baryon_type = splitted[3]


    run_dict = {
        "run name": run_name,
        "DM type": dm_type,
        "baryon_type": baryon_type
    }

    if dm_type == 'CDM':
        otherInfo = splitted[4:]
        run_dict['otherInfo'] = otherInfo
    elif (dm_type == '2cDM') or (dm_type == 'SIDM'):
        powerType = splitted[4]
        powerLaws = parsePower(powerType)
        sigmaType = splitted[5]
        sigma = parseSigma(sigmaType)
        otherInfo = splitted[6:]
        temp = {
            "powerLaws": powerLaws,
            "sigma0": sigma,
            "otherInfo": otherInfo
        }

        for o in otherInfo:
            if 'Vkick' in o: temp['Vkick'] = float(o.split('Vkick')[1])

        run_dict.update(temp)

    with h5py.File(snaps[0], 'r') as f:
        boxSize = int((f['Header'].attrs['BoxSize'])) # in kpc
        NPart = int(np.cbrt(f['Header'].attrs['NumPart_Total'][1]))
        mass_resolution = [f['Header'].attrs['MassTable'][1] * 10**10] # in solar masses
        if baryon_type == 'HY':
            masses = f.get('PartType0/Masses')[()]
            if np.all(np.isclose(masses, masses[0])):
                mass_resolution.append(masses[0] * 10**10)
        softening_length = boxSize / NPart / 29
        temp = {
            "BoxSize": boxSize,
            "NPart": NPart,
            "Mass Resolution": mass_resolution,
            "Softening Length": softening_length
        }

        run_dict.update(temp)


    redshifts = []
    omega0s = []
    file_indices = []
    for fof in fofs:
        i = int(Path(fof).stem.split('_')[-1])
        file_indices.append(i)
        with h5py.File(fof, 'r') as f:

            redshifts.append( float("{:.2f}".format(f['Header'].attrs['Redshift'])) )
            omega0s.append(f['Header'].attrs['Omega0'])
            try:
                subhalo = f.get('Subhalo')
                (mbins, mcount), (vbins, vcount) = get_profiles(f)
                tname = os.path.join(fpath, "mass_profile_{}.txt".format(i))
                np.savetxt(tname, (mbins, mcount))
                tname = os.path.join(fpath, "vel_profile_{}.txt".format(i))
                np.savetxt(tname, (vbins, vcount))
                tname = os.path.join(fpath, "subhalo_stats_{}.txt".format(i))
                np.savetxt(tname, ( subhalo['SubhaloVmax'], subhalo['SubhaloVmaxRad'], subhalo['SubhaloMass'], subhalo['SubhaloHalfmassRad'], subhalo['SubhaloMassInHalfRad'], subhalo['SubhaloMassInRad'], subhalo['SubhaloLen'] ) )

            except KeyError:
                continue
    run_dict['redshifts'] = redshifts
    run_dict['file_indices'] = file_indices
    run_dict['Omega0'] = omega0s
    if baryon_type == 'HY':
        omegaBs = []
        omegaStars = []
        boxVol_Mpc = (boxSize / 1000)**3

        for snap in snaps:
            with h5py.File(snap, 'r') as f:
                omegaBs.append(f.get('PartType0/Masses')[()].sum() * 10**10 / boxVol_Mpc / rho_crit)
                try:
                    omegaStars.append(f.get('PartType4/Masses')[()].sum() * 10**10 / boxVol_Mpc / rho_crit)
                except:
                    pass
        temp = {
            'OmegaB': omegaBs,
            'OmegaStar': omegaStars
        }
        run_dict.update(temp)


    fname = os.path.join(fpath, "run_info.json")
    with open(fname, 'w') as outfile:
        json.dump(run_dict, outfile)

  0%|          | 0/36 [00:00<?, ?it/s]

../../data_prods/run_2cDM_L3N128_HY_power00_sigma1 already exists!
Working on  run_2cDM_L3N128_HY_power00_sigma1
../../data_prods/run_2cDM_L3N256_HY_power00_sigma1 already exists!
Working on  run_2cDM_L3N256_HY_power00_sigma1
../../data_prods/run_2cDM_L3N256_HY_power00_sigma1_dir_0 already exists!
Working on  run_2cDM_L3N256_HY_power00_sigma1_dir_0
../../data_prods/run_2cDM_L3N256_HY_power00_sigma1_dir_1 already exists!
Working on  run_2cDM_L3N256_HY_power00_sigma1_dir_1
../../data_prods/run_2cDM_L3N256_HY_power00_sigma1_dir_2 already exists!
Working on  run_2cDM_L3N256_HY_power00_sigma1_dir_2
../../data_prods/run_2cDM_L3N256_HY_power00_sigma1_dir_3 already exists!
Working on  run_2cDM_L3N256_HY_power00_sigma1_dir_3
../../data_prods/run_2cDM_L3N256_HY_power00_sigma1_dir_4 already exists!
Working on  run_2cDM_L3N256_HY_power00_sigma1_dir_4
../../data_prods/run_2cDM_L3N256_HY_power00_sigma1_dir_5 already exists!
Working on  run_2cDM_L3N256_HY_power00_sigma1_dir_5
../../data_prods/run_2cD